In [ ]:
!pip install h2o


In [ ]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

In [ ]:
h2o.init(
    nthreads=-1,     # number of threads when launching a new H2O server
    max_mem_size=12  # in gigabytes
)

In [ ]:
train = h2o.import_file("../input/tabular-playground-series-aug-2021/train.csv")
test = h2o.import_file("../input/tabular-playground-series-aug-2021/test.csv")

In [ ]:
train_as_df = h2o.as_list(train,use_pandas=True)

round((train_as_df.isnull().sum() * 100/ len(train_as_df)),2).sort_values(ascending=False)


In [ ]:
train_as_df.head(10)


In [ ]:
train = h2o.H2OFrame(train_as_df)


In [ ]:
x = train.columns
y = "loss"
x.remove(y)
x.remove("id")

In [ ]:
aml = H2OAutoML(max_models=15, seed=1)
aml.train(x=x, y=y, training_frame=train)

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
test = h2o.as_list(test,use_pandas=True)

round((test.isnull().sum() * 100/ len(test)),2).sort_values(ascending=False)


In [ ]:
test = h2o.H2OFrame(test)


In [ ]:
test.columns.remove("id")



In [ ]:
preds = aml.predict(test)


In [ ]:
df = test.cbind(preds)
df.head(2)

In [ ]:
df=df.rename(columns={'predict':'loss'})
df.head()

In [ ]:
submission = df[:, ["id", "loss"]]
submission.head()

In [ ]:
h2o.export_file(submission, path = "./submission.csv")
